# UVM Environment

In [1]:
from pyuvm import *
from tb_pkg import *
import random

class BaseEnv(uvm_env):
    @staticmethod
    def alu_op(A, B, op, is_dut = False):
        """TLM model of the TinyALU"""
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.NOP:
            result =  A
        elif op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        if is_dut:
            time.sleep(0.1)
            if (random.randint(0,3) == 0):
                result = result + 1
        return result

    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")

class RandomEnv(BaseEnv):
    def run_phase(self):
        self.raise_objection() 
        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            self.cvg.add(op)
            predicted_result = self.alu_op(A, B, op)
            actual_result = self.alu_op(A, B, op, is_dut=True)
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
        self.drop_objection()  ## drop the objection to end

class MaxEnv(BaseEnv):
    def run_phase(self):
        self.raise_objection()
        A = 0xFF
        B = 0xFF
        for op in list(Ops):
            self.cvg.add(op)
            predicted_result = self.alu_op(A, B, op)
            actual_result = self.alu_op(A, B, op, is_dut=True)
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
        self.drop_objection()

class RandomTest(uvm_test):
    def build_phase(self):
        self.env = RandomEnv("env", self)

class MaxTest(uvm_test):
    def build_phase(self):
        self.env = MaxEnv("env", self)

# Be sure to restart the notebook between trying the two kinds of tests
# uvm_root().run_test("RandomTest")
uvm_root().run_test("MaxTest")


INFO: <ipython-input-1-6ffad4047489>(59)[uvm_test_top.env]: PASSED: ff NOP ff = 00ff
ERROR: <ipython-input-1-6ffad4047489>(61)[uvm_test_top.env]: FAILED: ff ADD ff = 01ff expected 01fe
INFO: <ipython-input-1-6ffad4047489>(59)[uvm_test_top.env]: PASSED: ff AND ff = 00ff
INFO: <ipython-input-1-6ffad4047489>(59)[uvm_test_top.env]: PASSED: ff XOR ff = 0000
INFO: <ipython-input-1-6ffad4047489>(59)[uvm_test_top.env]: PASSED: ff MUL ff = fe01
